In [45]:
!pip install python-dotenv

In [46]:
# Importing libraries
from dotenv import load_dotenv
import os
from langchain.schema import HumanMessage, SystemMessage
from IPython.display import display, Markdown
import google.generativeai as genai
from PIL import Image
import openai
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
import requests
from io import BytesIO

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
from dotenv import load_dotenv
import os

In [49]:
# Load the environment variables from the .env file
load_dotenv("/content/drive/My Drive/alsapikeys.env")

# Get the Gemini API Key and Open API Key from the environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Configure APIs
genai.configure(api_key=GEMINI_API_KEY)
openai.api_key = OPENAI_API_KEY

In [50]:
# Dataset Preprocessing
# (No code to correct, just a comment)

In [51]:
# Load and preprocess dataset
df = pd.read_csv("/content/drive/My Drive/fitness_data.csv")

# Filter relevant columns and clean data
df_cleaned = df[['Age', 'Gender', 'Resting_BPM']].dropna()
df_cleaned.to_csv("cleaned_fitness_data.csv", index=False)

# Prepare data for ML model
X = df_cleaned[['Age']]
y = df_cleaned['Resting_BPM']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
performance_metrics = classification_report(y_test, y_pred, output_dict=True)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [52]:
def generate_personal_plan(preferences):
    """Use Google Gemini to generate a personalized plan."""
    prompt = f"Create a workout and meal plan for someone who wants to {preferences['fitness_goal']}."
    if preferences.get("celebrity_preference"):
        prompt += f" Include workouts inspired by {preferences['celebrity_preference']}."
    try:
        response = genai.generate_content(prompt)
        return response.text
    except Exception as e:
        # Return a safe default prompt instead of the error message
        return f"Generate a visual representation of a {preferences['fitness_goal']} plan."

In [53]:
def generate_image(prompt, model="dall-e-3", size="1024x1024"):
    """Generate an image using OpenAI's DALL-E API."""
    # The openai.Image.create method has been removed.
    # Use openai.images.generate instead.
    response = openai.images.generate(
        prompt=prompt,
        model=model,
        size=size,
        n=1,
        response_format="url"
    )
    return response.data[0].url # Access data through the data attribute.

In [54]:
def get_user_preferences():
    """Collect user preferences."""
    fitness_goal = input("Enter your fitness goal (e.g., 'build muscle', 'lose weight'): ")
    celebrity_preference = input("Enter a celebrity's name for workout inspiration (or leave blank): ")
    return {"fitness_goal": fitness_goal, "celebrity_preference": celebrity_preference}

def fetch_workout_data(goal, celebrity=None):
    """Fetch workout plan based on goal and celebrity preference."""
    prompt = f"Create a workout plan for someone whose goal is to {goal}."
    if celebrity:
        prompt += f" Include workouts inspired by {celebrity}."
    return generate_personal_plan({"fitness_goal": goal, "celebrity_preference": celebrity})

def fetch_meal_plan(goal):
    """Fetch meal plan based on fitness goal."""
    return generate_personal_plan({"fitness_goal": goal})

def display_results_with_images(workout, meal, recommendation):
    """Display workout, meal plans, AI recommendations, and visuals."""
    workout_image_url = generate_image(workout)
    meal_image_url = generate_image(meal)

    display(Markdown(f"## Workout Plan\n{workout}"))
    display(Markdown(f"![Workout Image]({workout_image_url})"))
    display(Markdown(f"## Meal Plan\n{meal}"))
    display(Markdown(f"![Meal Image]({meal_image_url})"))
    display(Markdown(f"## AI Recommendation\n{recommendation}"))


In [55]:
!pip install --upgrade openai

In [56]:
# Get user preferences
user_preferences = get_user_preferences()

# Generate workout and meal plans
workout_plan = fetch_workout_data(user_preferences['fitness_goal'], user_preferences.get('celebrity_preference'))
meal_plan = fetch_meal_plan(user_preferences['fitness_goal'])

# Generate recommendations using the trained ML model
predicted_bpm = model.predict(pd.DataFrame([[30]], columns=['Age']))[0]  # Example age input
recommendation = f"Based on your age and resting BPM prediction ({predicted_bpm}), focus on a balanced routine with regular breaks."

# Display results with generated visuals
display_results_with_images(workout_plan, meal_plan, recommendation)

Enter your fitness goal (e.g., 'build muscle', 'lose weight'): lose weight
Enter a celebrity's name for workout inspiration (or leave blank): kate upton


## Workout Plan
Generate a visual representation of a lose weight plan.

![Workout Image](https://oaidalleapiprodscus.blob.core.windows.net/private/org-RyshVI88xm7djdYfwjL6AulT/user-9RiMt9bA1SgUJYqe6OP2QkLg/img-5vt7WawUVKQhzoloTTfux5tT.png?st=2024-12-28T18%3A32%3A24Z&se=2024-12-28T20%3A32%3A24Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-12-27T20%3A55%3A39Z&ske=2024-12-28T20%3A55%3A39Z&sks=b&skv=2024-08-04&sig=Og9pU3H%2BHUXT4L/kc50PWIS6f1E0WoxylXh49GCwfLY%3D)

## Meal Plan
Generate a visual representation of a lose weight plan.

![Meal Image](https://oaidalleapiprodscus.blob.core.windows.net/private/org-RyshVI88xm7djdYfwjL6AulT/user-9RiMt9bA1SgUJYqe6OP2QkLg/img-aMLSK6y50c68epexxnw7aQ0e.png?st=2024-12-28T18%3A32%3A37Z&se=2024-12-28T20%3A32%3A37Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-12-27T21%3A00%3A13Z&ske=2024-12-28T21%3A00%3A13Z&sks=b&skv=2024-08-04&sig=SSS9t7Ht1A3Rcpm/DtK4/XjF9PSfCsKTQISEOQ65fKQ%3D)

## AI Recommendation
Based on your age and resting BPM prediction (74.0), focus on a balanced routine with regular breaks.